# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846778817982                   -0.70    4.8
  2   -7.852312176244       -2.26       -1.53    1.0
  3   -7.852646202295       -3.48       -2.52    3.2
  4   -7.852646677473       -6.32       -3.37    2.0
  5   -7.852646685816       -8.08       -4.75    1.2
  6   -7.852646686723       -9.04       -5.18    3.8
  7   -7.852646686729      -11.21       -6.00    1.0
  8   -7.852646686730      -12.37       -7.18    2.8
  9   -7.852646686730   +    -Inf       -7.43    2.2
 10   -7.852646686730      -15.05       -8.57    2.0
 11   -7.852646686730   +  -14.75       -8.93    2.2
 12   -7.852646686730      -14.57       -9.13    1.0
 13   -7.852646686730   +    -Inf      -10.31    2.0
 14   -7.852646686730      -14.75      -10.49    1.2
 15   -7.852646686730      -14.75      -11.03    1.8
 16   -7.852646686730   +  -14.45      -11.47    1.8
 17   -7.852646686730   +  -14.75      -12.66 

## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846757701511                   -0.70           4.5
  2   -7.852526265321       -2.24       -1.63   0.80    2.2
  3   -7.852636735567       -3.96       -2.71   0.80    1.0
  4   -7.852646458412       -5.01       -3.27   0.80    2.2
  5   -7.852646676537       -6.66       -4.10   0.80    1.2
  6   -7.852646686621       -8.00       -4.76   0.80    2.2
  7   -7.852646686722      -10.00       -5.66   0.80    1.0
  8   -7.852646686730      -11.10       -6.80   0.80    2.2
  9   -7.852646686730      -12.65       -7.34   0.80    2.5
 10   -7.852646686730      -14.57       -8.30   0.80    1.2
 11   -7.852646686730   +  -15.05       -9.32   0.80    2.2
 12   -7.852646686730   +  -14.57      -10.40   0.80    2.5
 13   -7.852646686730      -15.05      -10.80   0.80    2.8
 14   -7.852646686730   +  -15.05      -11.99   0.80    1.0
 15   -7.852646686730      -14.75      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.396489e+01     3.624885e+00
 * time: 0.5809459686279297
     1     1.266791e+00     1.881944e+00
 * time: 0.8338849544525146
     2    -1.358369e+00     2.185131e+00
 * time: 0.8627939224243164
     3    -3.808380e+00     2.031262e+00
 * time: 0.9017770290374756
     4    -5.040150e+00     1.742657e+00
 * time: 0.9412789344787598
     5    -6.825720e+00     1.085950e+00
 * time: 0.9806840419769287
     6    -6.828447e+00     1.254849e+00
 * time: 1.008310079574585
     7    -7.457005e+00     7.755490e-01
 * time: 1.0362391471862793
     8    -7.501660e+00     8.261294e-01
 * time: 1.0660901069641113
     9    -7.756270e+00     3.757554e-01
 * time: 1.0960609912872314
    10    -7.818225e+00     1.549047e-01
 * time: 1.1251821517944336
    11    -7.837388e+00     6.985469e-02
 * time: 1.153581142425537
    12    -7.844050e+00     6.131709e-02
 * time: 1.182049036026001
    13    -7.845898e+00     4.637303e-02
 * time: 1.2100110054016

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846771342125                   -0.70    4.5
  2   -7.852298979707       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ, _ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation)
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686662                   -2.54
  2   -7.852646686730      -10.17       -5.80
  3   -7.852646686730   +    -Inf      -12.09


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 8.291692335274598e-14
|ρ_newton - ρ_scfv| = 6.806480131407212e-14
|ρ_newton - ρ_dm|   = 8.637278435167049e-10
